### Tensor flow with scikit learn using tf.learn (classfier and regressor)  
Note: changes to tf.learn API cause warnings and SKCompat is not fully functional

In [223]:
import tensorflow as tf 
#tf.__version__   # '1.1.0'

# we use this library to deal with files and folders
import shutil

# Cleans the given folder if it exists (model remembers past training and will not work if layers change)
def clean_folder(folder):
    try:
        shutil.rmtree(folder)
    except OSError:
        pass

In [224]:
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, classification_report
from sklearn.metrics import confusion_matrix , mean_squared_error, r2_score
from sklearn.datasets import load_iris, load_boston
iris = load_iris()
X = iris.data
y = iris.target 
X = X.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 0.7, random_state=707)

#val_monitor = tf.contrib.learn.monitors.ValidationMonitor(X_test, y_test, early_stopping_rounds=200)
# Define the inputs for new api
def get_train_inputs():
    x = tf.constant(X_train)
    y = tf.constant(y_train)
    return x, y
# Define the test inputs
def get_test_inputs():
    x = tf.constant(X_test)
    y = tf.constant(y_test)
    return x, y

In [225]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
# can also pecify that all features have real-value data
#feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

iris_dir = '/tmp/iris_model'
clean_folder(iris_dir)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[4,4,4], n_classes= 3,
                                         feature_columns=feature_columns,
                                            model_dir=iris_dir)
#dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_task_type': None, '_master': '', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_model_dir': None, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022D81906EF0>, '_save_checkpoints_steps': None, '_is_chief': True}


In [226]:
#dnn_clf.fit(input_fn=get_train_inputs, steps=200, monitors=[val_monitor])
dnn_clf.fit(input_fn=get_train_inputs, steps=200) 
# batch_size=128, steps=None, max_steps=None, monitors=None
#dnn_clf.fit(X_train, y_train, steps=None, max_steps= 500)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:step = 1, loss = 1.15226
INFO:tensorflow:global_step/sec: 925.772
INFO:tensorflow:step = 101, loss = 0.588292 (0.111 sec)
INFO:tensorflow:Saving checkpoints for 200 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:Loss for final step: 0.319013.


DNNClassifier(params={'activation_fn': <function relu at 0x0000022DEB02A048>, 'dropout': None, 'hidden_units': [4, 4, 4], 'gradient_clip_norm': None, 'input_layer_min_slice_size': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x0000022D81DF1240>, 'optimizer': None, 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'embedding_lr_multipliers': None})

In [227]:
# Evaluate accuracy.
accuracy_score = dnn_clf.evaluate(input_fn=get_test_inputs,
                                     steps=1)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-04-15:01:25
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-200
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-05-04-15:01:25
INFO:tensorflow:Saving dict for global step 200: accuracy = 1.0, global_step = 200, loss = 0.268876

Test Accuracy: 1.000000



In [228]:
y_pred = list(dnn_clf.predict_classes(input_fn = get_test_inputs))
#accuracy = accuracy_score(y_test, y_pred)
#accuracy

INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-200


In [229]:
y_pred_proba = list(dnn_clf.predict_proba(input_fn = get_test_inputs))
log_loss(y_test, y_pred_proba)

INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-200


0.26887595396902825

In [230]:
print("Confusion matrix:\n", confusion_matrix(y_test,y_pred))
print("Classification report:\n", classification_report(y_test,y_pred))

Confusion matrix:
 [[15  0  0]
 [ 0 15  0]
 [ 0  0 15]]
Classification report:
              precision    recall  f1-score   support

          0       1.00      1.00      1.00        15
          1       1.00      1.00      1.00        15
          2       1.00      1.00      1.00        15

avg / total       1.00      1.00      1.00        45



### Regressor using Boston dataset

In [234]:
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
boston = load_boston()
Xb, yb = shuffle(boston.data, boston.target, random_state=13)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, train_size=0.75, random_state=717)
scaler = StandardScaler()
scaler.fit(Xb_train)
Xb_train = scaler.transform(Xb_train)
Xb_test = scaler.transform(Xb_test)
def get_train_inputs_b():
    x = tf.constant(Xb_train)
    y = tf.constant(yb_train)
    return x, y
# Define the test inputs
def get_test_inputs_b():
    x = tf.constant(Xb_test)
    y = tf.constant(yb_test)
    return x, y

In [288]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(Xb_train)

boston_dir = '/tmp/boston_model'
clean_folder(boston_dir)

dnn_reg = tf.contrib.learn.DNNRegressor(
      feature_columns=feature_columns, hidden_units=[20, 10, 10], model_dir = boston_dir, 
optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001),
    activation_fn = tf.nn.relu6
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_task_type': None, '_master': '', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_model_dir': None, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022D85AD2C50>, '_save_checkpoints_steps': None, '_is_chief': True}


In [289]:
# Fit model
dnn_reg.fit(input_fn = get_train_inputs_b, steps=400)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/boston_model\model.ckpt.
INFO:tensorflow:step = 1, loss = 582.742
INFO:tensorflow:global_step/sec: 721.905
INFO:tensorflow:step = 101, loss = 11.0498 (0.140 sec)
INFO:tensorflow:global_step/sec: 775.059
INFO:tensorflow:step = 201, loss = 5.72366 (0.129 sec)
INFO:tensorflow:global_step/sec: 858.068
INFO:tensorflow:step = 301, loss = 4.54546 (0.117 sec)
INFO:tensorflow:Saving checkpoints for 400 into /tmp/boston_model\model.ckpt.
INFO:tensorflow:Loss for final step: 3.69198.


DNNRegressor(params={'activation_fn': <function relu6 at 0x0000022DEB323A60>, 'dropout': None, 'hidden_units': [20, 10, 10], 'gradient_clip_norm': None, 'input_layer_min_slice_size': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x0000022D85AD2668>, 'optimizer': <tensorflow.python.training.proximal_adagrad.ProximalAdagradOptimizer object at 0x0000022D890EF0F0>, 'feature_columns': (_RealValuedColumn(column_name='', dimension=13, default_value=None, dtype=tf.float64, normalizer=None),), 'embedding_lr_multipliers': None})

In [290]:
# Predict and score
yb_predicted = list(dnn_reg.predict_scores(input_fn = get_test_inputs_b, as_iterable=True))
score = mean_squared_error(yb_predicted, yb_test)
Rsq = r2_score(yb_test, yb_predicted)
print('MSE: {0:f}'.format(score))
print('R^2: {0:f}'.format(Rsq))

INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-400
MSE: 7.208418
R^2: 0.913897
